In [74]:
import numpy as np
import scipy as sp
import pandas as pd

In [75]:
# get data
df = pd.read_csv('high_diamond_ranked_10min.csv')

# print all columns
for col in df.columns:
    print(col)

gameId
blueWins
blueWardsPlaced
blueWardsDestroyed
blueFirstBlood
blueKills
blueDeaths
blueAssists
blueEliteMonsters
blueDragons
blueHeralds
blueTowersDestroyed
blueTotalGold
blueAvgLevel
blueTotalExperience
blueTotalMinionsKilled
blueTotalJungleMinionsKilled
blueGoldDiff
blueExperienceDiff
blueCSPerMin
blueGoldPerMin
redWardsPlaced
redWardsDestroyed
redFirstBlood
redKills
redDeaths
redAssists
redEliteMonsters
redDragons
redHeralds
redTowersDestroyed
redTotalGold
redAvgLevel
redTotalExperience
redTotalMinionsKilled
redTotalJungleMinionsKilled
redGoldDiff
redExperienceDiff
redCSPerMin
redGoldPerMin


In [76]:
df.head()

,gameId,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,...,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
0,4519157822,0,28,2,1,9,6,11,0,0,...,0,16567,6.8,17047,197,55,-643,8,19.7,1656.7
1,4523371949,0,12,1,0,5,5,5,0,0,...,1,17620,6.8,17438,240,52,2908,1173,24.0,1762.0
2,4521474530,0,15,0,0,7,11,4,1,1,...,0,17285,6.8,17254,203,28,1172,1033,20.3,1728.5
3,4524384067,0,43,1,0,4,5,5,1,0,...,0,16478,7.0,17961,235,47,1321,7,23.5,1647.8
4,4436033771,0,75,4,0,6,6,6,0,0,...,0,17404,7.0,18313,225,67,1004,-230,22.5,1740.4


In [77]:
# Check if there are redDeaths that are not equal to blueKills
diff = np.where(df['redDeaths'] != df['blueKills'], df['blueKills'], None)
diff = diff[diff != None]
diff.size

0

In [78]:
# Check if there are blueDeaths that are not equal to redKills
diff = np.where(df['blueDeaths'] != df['redKills'], df['redKills'], None)
diff = diff[diff != None]
diff.size

0

In [79]:
# redExperienceDiff is just the negative of blueExperienceDiff
df[['redExperienceDiff', 'blueExperienceDiff']].head()

,redExperienceDiff,blueExperienceDiff
0,8,-8
1,1173,-1173
2,1033,-1033
3,7,-7
4,-230,230


In [80]:
# redGoldDiff is just the negative of blueGoldDiff
df[['redGoldDiff', 'blueGoldDiff']].head()

,redGoldDiff,blueGoldDiff
0,-643,643
1,2908,-2908
2,1172,-1172
3,1321,-1321
4,1004,-1004


In [81]:
df[['redFirstBlood', 'blueFirstBlood']].head()

,redFirstBlood,blueFirstBlood
0,0,1
1,1,0
2,1,0
3,1,0
4,1,0


In [82]:
df[['blueEliteMonsters', 'blueDragons', 'blueHeralds', 'blueTotalMinionsKilled', 'blueTotalJungleMinionsKilled']]

,blueEliteMonsters,blueDragons,blueHeralds,blueTotalMinionsKilled,blueTotalJungleMinionsKilled
0,0,0,0,195,36
1,0,0,0,174,43
2,1,1,0,186,46
3,1,0,1,201,55
4,0,0,0,210,57
...,...,...,...,...,...
9874,1,1,0,211,69
9875,1,1,0,233,48
9876,0,0,0,210,45
9877,1,1,0,224,48


In [83]:
df[['blueTotalExperience', 'redTotalExperience']].head()

,blueTotalExperience,redTotalExperience
0,17039,17047
1,16265,17438
2,16221,17254
3,17954,17961
4,18543,18313


In [84]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
df = shuffle(df)

In [85]:
# feature selection 1
# use random forest to sort features by 'importance'

# select all features
x = df.iloc[:, 2:]
y = df.to_numpy()[:, 1]
y = np.reshape(y, (y.shape[0],))

n = y.shape[0]
split = int(n*0.8) # 80/20 split

# train random forest
clf = RandomForestClassifier()
clf.fit(x.iloc[:split, :], y[:split])
print("Overall score: {:.4f}\n".format(clf.score(x.iloc[split:, :], y[split:])))

features = list(x.columns)
feature_ranks = sorted(zip(clf.feature_importances_, features), reverse=True)

for importance, feature in feature_ranks:
    print("{:.4f}\t{}".format(importance, feature))

Overall score: 0.7161

0.0894	redGoldDiff
0.0845	blueGoldDiff
0.0758	blueExperienceDiff
0.0719	redExperienceDiff
0.0469	blueGoldPerMin
0.0459	blueTotalGold
0.0419	redGoldPerMin
0.0411	redTotalGold
0.0365	redTotalExperience
0.0355	blueTotalExperience
0.0276	redTotalJungleMinionsKilled
0.0273	blueTotalJungleMinionsKilled
0.0271	blueWardsPlaced
0.0270	blueTotalMinionsKilled
0.0270	redCSPerMin
0.0269	blueCSPerMin
0.0269	redTotalMinionsKilled
0.0255	redWardsPlaced
0.0222	blueAssists
0.0210	redAssists
0.0183	blueWardsDestroyed
0.0179	redWardsDestroyed
0.0177	blueDeaths
0.0163	blueKills
0.0147	redKills
0.0147	blueAvgLevel
0.0143	redDeaths
0.0125	redAvgLevel
0.0079	blueEliteMonsters
0.0078	blueDragons
0.0069	redEliteMonsters
0.0060	redDragons
0.0041	redFirstBlood
0.0038	blueFirstBlood
0.0034	blueHeralds
0.0033	redHeralds
0.0011	blueTowersDestroyed
0.0011	redTowersDestroyed


In [86]:
# feature selection 2
# use random forest to sort features by 'importance'

# select only features of blue team
blue_wins = np.where(df.columns == 'blueWins', True, False)

x = df.iloc[:, 2:20]
y = df.to_numpy()[:, blue_wins]
y = np.reshape(y, (y.shape[0],))

# train random forest
clf = RandomForestClassifier()
clf.fit(x.iloc[:split, :], y[:split])
print("Overall score: {:.4f}\n".format(clf.score(x.iloc[split:, :], y[split:])))

features = list(x.columns)
feature_ranks = sorted(zip(clf.feature_importances_, features), reverse=True)

for importance, feature in feature_ranks:
    print("{:.4f}\t{}".format(importance, feature))

Overall score: 0.7095

0.1736	blueGoldDiff
0.1497	blueExperienceDiff
0.1078	blueTotalGold
0.0976	blueTotalExperience
0.0567	blueTotalJungleMinionsKilled
0.0551	blueTotalMinionsKilled
0.0545	blueWardsPlaced
0.0537	blueCSPerMin
0.0506	blueDeaths
0.0450	blueAssists
0.0403	blueKills
0.0367	blueWardsDestroyed
0.0298	blueAvgLevel
0.0138	blueEliteMonsters
0.0137	blueDragons
0.0116	blueFirstBlood
0.0072	blueHeralds
0.0026	blueTowersDestroyed


In [87]:
# feature selection 2
# use random forest to sort features by 'importance'

# select only features of blue team without gold  or experience diff
# this way, we don't rely on any data from the red team
blue_wins = np.where(df.columns == 'blueWins', True, False)

x = df.iloc[:, 2:20]
x = x.drop(columns=['blueGoldDiff', 'blueExperienceDiff'])
y = df.to_numpy()[:, blue_wins]
y = np.reshape(y, (y.shape[0],))

# train random forest
clf = RandomForestClassifier()
clf.fit(x.iloc[:split, :], y[:split])
print("Overall score: {:.4f}\n".format(clf.score(x.iloc[split:, :], y[split:])))

features = list(x.columns)
feature_ranks = sorted(zip(clf.feature_importances_, features), reverse=True)

for importance, feature in feature_ranks:
    print("{:.4f}\t{}".format(importance, feature))

Overall score: 0.7029

0.1742	blueTotalGold
0.1430	blueTotalExperience
0.0986	blueDeaths
0.0778	blueTotalJungleMinionsKilled
0.0753	blueTotalMinionsKilled
0.0738	blueCSPerMin
0.0720	blueWardsPlaced
0.0659	blueKills
0.0626	blueAssists
0.0492	blueWardsDestroyed
0.0442	blueAvgLevel
0.0189	blueDragons
0.0172	blueEliteMonsters
0.0153	blueFirstBlood
0.0089	blueHeralds
0.0033	blueTowersDestroyed
